# TiDB Todo Branch Storyboard (Agent-driven)

This notebook drives the branching workflow **via prompts to the `tidb_agent`** instead of hardcoded SQL. The agent has tools to create/manage branches and to run SQL on specific branches (`run_sql_on_branch`). We'll ask it to propose schemas, apply them per-branch, and then inspect/visualize the results.

## Setup

In [ ]:
%pip install -q -r requirements.txt

In [ ]:
import json
import pandas as pd
from IPython.display import Markdown, display

from tidb_agent import agent, branch_manager, select_branch_rows

pd.set_option("display.max_colwidth", 160)

## Step 1: Ask the agent to draft three competing todo schemas (three branches)

Edit/run the prompt below. The agent should use `create_branch` and `run_sql_on_branch` to build three approaches and seed sample data.

In [ ]:
stage1_prompt = """
You are the TiDB agent. Design a new todo app schema via branching with a hard cap of 5 total branches in this free-tier cluster:
- Create exactly three branches off the default branch named todo_minimal, todo_tags, and todo_checklists. If they exist, reset/reuse as needed.
- Do NOT create backups (branch budget is tight).
- On each branch, propose a distinct schema approach for todos (minimal single table, tags join table, checklist items with reminders) and apply it using run_sql_on_branch.
- Seed a few representative rows per branch.
- Return a concise summary and example rows by selecting from each table.
Use create_branch for branching (or create_branch_from_display_name if you need a parent). Use run_sql_on_branch to apply DDL/DML on the correct branch.
Send SQL as single statements only (multi-statement batches are disabled); create tables before altering them.
"""

stage1_result = agent(stage1_prompt)
print(stage1_result)
if stage1_result.metrics:
    display(Markdown("**Tool metrics**"))
    display(stage1_result.metrics.get_summary())

### Inspect the three base branches

Run following code block, and pick a branch by ENTER `1`, `2`, or `3` when prompted from the top dialog.

In [ ]:
# Discover branches created in step 1; fall back to all branches if filter yields none.
all_branches = branch_manager.list_branches()
base_names = [b.get("displayName") for b in all_branches if b.get("displayName", "").startswith("todo_")]
if not base_names:
    base_names = [b.get("displayName") for b in all_branches]

branch_id_map = {b.get("displayName"): b.get("branchId") for b in all_branches}
for idx, name in enumerate(base_names, start=1):
    display(Markdown(f"### {idx}. `{name}` (id: {branch_id_map.get(name)})"))
    try:
        tables = select_branch_rows(name, "SHOW TABLES")
        display(Markdown("**Tables**"))
        display(pd.DataFrame(tables))
        if tables:
            first_table = list(tables[0].values())[0]
            rows = select_branch_rows(name, f"SELECT * FROM {first_table} LIMIT 20")
            display(Markdown(f"**Sample rows from `{first_table}`**"))
            display(pd.DataFrame(rows))
    except Exception as exc:
        display(Markdown(f"Error inspecting branch `{name}`: {exc}"))
    display(Markdown("---"))

if base_names:
    try:
        choice_raw = input(f"Select base (1-{len(base_names)}), default=1: ").strip() or "1"
        choice_idx = int(choice_raw)
        if 1 <= choice_idx <= len(base_names):
            selected_base = base_names[choice_idx - 1]
        else:
            selected_base = base_names[0]
    except Exception:
        selected_base = base_names[0]
else:
    selected_base = ""
selected_base_id = branch_id_map.get(selected_base)
display(Markdown(f"Using base: `{selected_base}` (id: {selected_base_id})"))

## Step 2: Branch from your chosen base to add users

Update the prompt to reference the selected base branch. The agent should create three variations (e.g., single owner, many-to-many assignments, audited) using `run_sql_on_branch`.

In [ ]:
selected_base = selected_base  # from prior cell
selected_base_id = selected_base_id  # from prior cell
stage2_prompt = f"""
You are the TiDB agent. The free-tier cluster only allows 5 total branches. We already spent 3 above, so you may create at most 2 more.
Starting from parent branch `{selected_base}` (id `{selected_base_id}`), create exactly two child branches:
- `{selected_base}_owners`: add a users table and a single owner per todo.
- `{selected_base}_assignments`: add a users table and a many-to-many todo_assignees join table.
For each branch: create it using create_branch_from_display_name (preferred) with parent `{selected_base}` (id `{selected_base_id}`), then apply schema and seed rows via run_sql_on_branch, and show sample selects.
Do NOT create these from main/default; always branch from `{selected_base}`. If branching fails, stop and report instead of creating off main. Do NOT create backups (branch budget is tight).
Send SQL as single statements only (multi-statement batches are disabled); create tables before altering them.
Return a brief summary of changes and sample data per branch.
"""

if selected_base and selected_base_id:
    stage2_result = agent(stage2_prompt)
    print(stage2_result)
    if stage2_result.metrics:
        display(Markdown("**Tool metrics**"))
        display(stage2_result.metrics.get_summary())
else:
    print("No base branch selected; skip stage 2 prompt.")

all_branches = branch_manager.list_branches()child_candidates = [b.get("displayName") for b in all_branches if selected_base and b.get("displayName", "").startswith(selected_base)]child_names = [name for name in child_candidates if name != selected_base]child_id_map = {b.get("displayName"): b.get("branchId") for b in all_branches}if not child_names:    child_names = [b.get("displayName") for b in all_branches]for idx, name in enumerate(child_names, start=1):    display(Markdown(f"### {idx}. `{name}` (id: {child_id_map.get(name)})"))    try:        tables = select_branch_rows(name, "SHOW TABLES")        display(Markdown("**Tables**"))        display(pd.DataFrame(tables))        if tables:            first_table = list(tables[0].values())[0]            rows = select_branch_rows(name, f"SELECT * FROM {first_table} LIMIT 20")            display(Markdown(f"**Sample rows from `{first_table}`**"))            display(pd.DataFrame(rows))    except Exception as exc:        display(Markdown(f"Error inspecting branch `{name}`: {exc}"))    display(Markdown("---"))if child_names:    try:        choice_raw = input(f"Select user branch (1-{len(child_names)}), default=1: ").strip() or "1"        choice_idx = int(choice_raw)        if 1 <= choice_idx <= len(child_names):            child_selected = child_names[choice_idx - 1]        else:            child_selected = child_names[0]    except Exception:        child_selected = child_names[0]else:    child_selected = ""display(Markdown(f"Inspecting user branch: `{child_selected}` (id: {child_id_map.get(child_selected)})"))

In [ ]:
all_branches = branch_manager.list_branches()
child_candidates = [b.get("displayName") for b in all_branches if selected_base and b.get("displayName", "").startswith(selected_base)]
child_names = [name for name in child_candidates if name != selected_base]
child_id_map = {b.get("displayName"): b.get("branchId") for b in all_branches}
if not child_names:
    child_names = [b.get("displayName") for b in all_branches]

for idx, name in enumerate(child_names, start=1):
    display(Markdown(f"### {idx}. `{name}` (id: {child_id_map.get(name)})"))
    try:
        tables = select_branch_rows(name, "SHOW TABLES")
        display(Markdown("**Tables**"))
        display(pd.DataFrame(tables))
        if tables:
            first_table = list(tables[0].values())[0]
            rows = select_branch_rows(name, f"SELECT * FROM {first_table} LIMIT 20")
            display(Markdown(f"**Sample rows from `{first_table}`**"))
            display(pd.DataFrame(rows))
    except Exception as exc:
        display(Markdown(f"Error inspecting branch `{name}`: {exc}"))
    display(Markdown("---"))

## Step 3: Visualize branch lineage

Graph of all branches from TiDB Cloud (left-to-right).

In [ ]:
branches = branch_manager.list_branches()
by_parent = {}
for b in branches:
    by_parent.setdefault(b.get("parentId"), []).append(b)
for kids in by_parent.values():
    kids.sort(key=lambda x: x.get("displayName") or "")
branch_ids = {b.get("branchId") for b in branches}
roots = [b for b in branches if b.get("parentId") not in branch_ids]
roots.sort(key=lambda x: x.get("displayName") or "")

lines = []

def render_sub(node, prefix):
    children = by_parent.get(node.get("branchId"), [])
    for idx, child in enumerate(children):
        is_last = idx == len(children) - 1
        connector = "└─ " if is_last else "├─ "
        child_label = f"{child.get('displayName')} ({(child.get('branchId') or '')[:8]})"
        lines.append(prefix + connector + child_label)
        render_sub(child, prefix + ("   " if is_last else "│  "))

for idx, root in enumerate(roots):
    root_label = f"{root.get('displayName')} ({(root.get('branchId') or '')[:8]})"
    lines.append(root_label)
    render_sub(root, "")

print("Branch tree:")
print("\n".join(lines))

## Notes

- You can tweak the prompts and re-run cells to iterate on the schemas.
- The agent now supports `run_sql_on_branch` (added in `tidb_agent.py`) so branch-specific DDL/DML flows through the correct connection.
- Consider adding performance checks (indexes, query plans) as follow-up prompts.